In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt


def read_and_plot(file_path, title, scopy_file=None):
    df = pd.read_csv(file_path, delimiter="\t", skiprows=1, encoding="ISO-8859-1", names=["Freq", "Magnitude"], dtype=str)
    df[['Magnitude_dB', 'Phase_deg']] = df['Magnitude'].str.extract(
        r'\(\s*(-?[0-9]*\.?[0-9]+(?:[eE][+-]?[0-9]+)?)dB,\s*(-?[0-9]*\.?[0-9]+(?:[eE][+-]?[0-9]+)?)°\s*\)'
    )
    df['Magnitude_dB'] = pd.to_numeric(df['Magnitude_dB'], errors='coerce')
    df['Phase_deg'] = pd.to_numeric(df['Phase_deg'], errors='coerce')
    df['Freq'] = pd.to_numeric(df['Freq'], errors='coerce')
    #df = df.dropna(subset=['Magnitude_dB', 'Phase_deg', 'Freq'])

    df_scopy = None  # Initialize Scopy dataframe as None

    fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    # LTSpice plots
    ax[0].semilogx(df['Freq'], df['Magnitude_dB'], label='LTSpice Gain (dB)', color='tab:green')
    ax[1].semilogx(df['Freq'], df['Phase_deg'], label='LTSpice Phase (deg)', color='tab:green')

    if scopy_file:
        df_scopy = pd.read_csv(scopy_file, skiprows=7, encoding="ISO-8859-1", delimiter=",")
        df_scopy['Frequency(Hz)'] = pd.to_numeric(df_scopy['Frequency(Hz)'])
        df_scopy['Magnitude(dB)'] = pd.to_numeric(df_scopy['Magnitude(dB)'])
        df_scopy['Phase(°)'] = pd.to_numeric(df_scopy['Phase(°)'])

        scopy_mag_interp = np.interp(df['Freq'], df_scopy['Frequency(Hz)'], df_scopy['Magnitude(dB)'])
        scopy_phase_interp = np.interp(df['Freq'], df_scopy['Frequency(Hz)'], df_scopy['Phase(°)'])

        ax[0].semilogx(df['Freq'], scopy_mag_interp, label='Scopy Measured Gain', color='tab:blue', linestyle='dashed')
        ax[1].semilogx(df['Freq'], scopy_phase_interp, label='Scopy Measured Phase', color='tab:red', linestyle='dashed')

    ax[0].set_ylabel("Magnitude (dB)")
    ax[1].set_ylabel("Phase (deg)")
    ax[1].set_xlabel("Frequency (Hz)")

    ax[0].legend()
    ax[1].legend()
    ax[0].grid(True, which="both", linestyle="--")
    ax[1].grid(True, which="both", linestyle="--")

    fig.suptitle(title)
    fig.tight_layout()
    plt.show()
    return df, df_scopy 

file_paths = {
    '/home/art/school/ece2001/reports/report6/circuit1.txt': {
        'title': 'Circuit 1, RC High-Pass Filter',
        'scopy': '/home/art/school/ece2001/reports/report6/circuit1.csv'
    },
    '/home/art/school/ece2001/reports/report6/circuit2.txt': {
        'title': 'Circuit 2, RC Low-Pass Filter',
        'scopy': '/home/art/school/ece2001/reports/report6/circuit2.csv'
    },
    '/home/art/school/ece2001/reports/report6/circuit3.txt': {
        'title': 'Circuit 3, RC Parallel',
        'scopy': '/home/art/school/ece2001/reports/report6/circuit3.csv'
    },
    '/home/art/school/ece2001/reports/report6/circuit4a.txt': {
        'title': 'Circuit 4a, R=200',
        'scopy': '/home/art/school/ece2001/reports/report6/circuit4.csv'
    },
    '/home/art/school/ece2001/reports/report6/circuit4b.txt': {
        'title': 'Circuit 4b, R=2000',
        #'scopy': '/home/art/school/ece2001/reports/report6/scopy4b.csv'
    },
}

dfs = {}
scopy_dfs = {}
for idx, (path, info) in enumerate(file_paths.items()):
    ltspice_df, scopy_df = read_and_plot(path, info['title'], scopy_file=info.get('scopy'))
    dfs[idx] = ltspice_df
    scopy_dfs[idx] = scopy_df  # Store Scopy dataframe separately

circuit1 = dfs[0]
circuit2 = dfs[1]
circuit3 = dfs[2]
circuit4a = dfs[3]
circuit4b = dfs[4]

scopy_circuit1 = scopy_dfs[0]
scopy_circuit2 = scopy_dfs[1]
scopy_circuit3 = scopy_dfs[2]
scopy_circuit4a = scopy_dfs[3]
scopy_circuit4b = scopy_dfs[4]




In [9]:
def summarize_by_decade(df_scopy, title):
    if df_scopy is None:
        print(f"\n{title}: No Scopy data available.")
        return

    freqs = df_scopy['Frequency(Hz)']
    fmin, fmax = freqs.min(), freqs.max()

    # Decades from min to max in base-10 log scale
    decades = np.logspace(np.floor(np.log10(fmin)), np.ceil(np.log10(fmax)), num=10)
    
    print(f"\n{title}")
    print(f"{'Freq (Hz)':>12} | {'Gain (dB)':>10} | {'Phase (°)':>10}")
    print("-" * 40)

    for target_freq in decades:
        idx = (np.abs(freqs - target_freq)).idxmin()
        freq_val = df_scopy.loc[idx, 'Frequency(Hz)']
        gain_val = df_scopy.loc[idx, 'Magnitude(dB)']
        phase_val = df_scopy.loc[idx, 'Phase(°)']
        print(f"{freq_val:12.2f} | {gain_val:10.3f} | {phase_val:10.3f}")
summarize_by_decade(scopy_circuit1, "Circuit 1: RC High-Pass Filter")
summarize_by_decade(scopy_circuit2, "Circuit 2: RC Low-Pass Filter")
summarize_by_decade(scopy_circuit3, "Circuit 3: RC Parallel")
summarize_by_decade(scopy_circuit4a, "Circuit 4a: R=200")



Circuit 1: RC High-Pass Filter
   Freq (Hz) |  Gain (dB) |  Phase (°)
----------------------------------------
        1.00 |    -81.680 |    123.927
        5.99 |    -54.359 |     86.142
       35.94 |    -41.073 |     86.300
      215.44 |    -26.350 |     85.221
     1291.55 |    -11.371 |     70.992
     7742.64 |     -1.522 |     30.026
    46415.90 |     -0.076 |      6.315
   278256.00 |      0.014 |      1.098
  1668100.00 |      0.024 |      0.200
 10000000.00 |      0.040 |     -0.450

Circuit 2: RC Low-Pass Filter
   Freq (Hz) |  Gain (dB) |  Phase (°)
----------------------------------------
        1.00 |     -0.045 |     -0.779
        5.99 |     -0.031 |     -0.058
       35.94 |     -0.036 |     -0.487
      215.44 |     -0.063 |     -2.738
     1291.55 |     -0.507 |    -15.575
     7742.64 |     -6.089 |    -56.304
    46415.90 |    -19.171 |    -78.441
   278256.00 |    -33.672 |    -78.289
  1668100.00 |    -49.547 |    -64.173
 10000000.00 |    -38.133 |    141.5

In [10]:
def tabulate_cutoff_values(df_scopy, title, target_freqs):
    if df_scopy is None:
        print(f"\n{title}: No Scopy data available.")
        return
    
    freqs = df_scopy['Frequency(Hz)']
    print(f"\n{title}")
    print(f"{'Target Freq (Hz)':>16} | {'Measured Freq (Hz)':>18} | {'Gain (dB)':>10} | {'Phase (°)':>10}")
    print("-" * 65)
    
    for target in target_freqs:
        # Find the index of the frequency closest to the target
        idx = (np.abs(freqs - target)).idxmin()
        measured_freq = df_scopy.loc[idx, 'Frequency(Hz)']
        measured_gain = df_scopy.loc[idx, 'Magnitude(dB)']
        measured_phase = df_scopy.loc[idx, 'Phase(°)']
        print(f"{target:16.2f} | {measured_freq:18.2f} | {measured_gain:10.3f} | {measured_phase:10.3f}")

# Now, based on your LTSpice simulated cutoff values, tabulate the corresponding Scopy values.

# Circuit One: RC High-Pass Filter (Cutoff Frequency: 3556.31 Hz, Phase: 45.49°)
tabulate_cutoff_values(scopy_circuit1, "Circuit 1: RC High-Pass Filter", [3556.31])

# Circuit Two: RC Low-Pass Filter (Cutoff Frequency: 3681.29 Hz, Phase: -45.50°)
tabulate_cutoff_values(scopy_circuit2, "Circuit 2: RC Low-Pass Filter", [3681.29])

# Circuit Three: Parallel Capacitor Low Pass Filter (Cutoff Frequency: 1377.21 Hz, Phase: -53.18°)
tabulate_cutoff_values(scopy_circuit3, "Circuit 3: RC Parallel", [1377.21])

# Circuit Four (Resistor at 200Ω): Bandpass Filter with two cutoff frequencies
tabulate_cutoff_values(scopy_circuit4a, "Circuit 4a: R=200", [5105.05, 5597.57])


Circuit 1: RC High-Pass Filter
Target Freq (Hz) | Measured Freq (Hz) |  Gain (dB) |  Phase (°)
-----------------------------------------------------------------
         3556.31 |            3569.05 |     -4.393 |     50.071

Circuit 2: RC Low-Pass Filter
Target Freq (Hz) | Measured Freq (Hz) |  Gain (dB) |  Phase (°)
-----------------------------------------------------------------
         3681.29 |            3686.10 |     -2.466 |    -37.314

Circuit 3: RC Parallel
Target Freq (Hz) | Measured Freq (Hz) |  Gain (dB) |  Phase (°)
-----------------------------------------------------------------
         1377.21 |            1377.65 |     -2.385 |    -42.622

Circuit 4a: R=200
Target Freq (Hz) | Measured Freq (Hz) |  Gain (dB) |  Phase (°)
-----------------------------------------------------------------
         5105.05 |            5089.87 |      3.675 |    -33.948
         5597.57 |            5607.23 |      4.286 |    -40.791
